In [1]:
import sys
from pathlib import Path

here = Path.cwd().parent.parent
sys.path.append(str(here))

from lionfuncs import flatten, extract_json_schema

In [2]:
a = {"a": [1, [[2]], [{3}]], "g": {"b": [4]}}


a1 = flatten(a)
print(a1)

{'a|0': 1, 'a|2|0': {3}, 'g|b|0': 4, 'a|1|0|0': 2}


In [3]:
a2 = flatten(a, dynamic=True, coerce_sequence="dict", coerce_keys=False)
a3 = flatten(a, dynamic=True, coerce_sequence="list", coerce_keys=False)
print(a2)
print(a3)

{('a', '0'): 1, ('a', '2', '0'): {3}, ('g', 'b', '0'): 4, ('a', '1', '0', '0'): 2}
{('a', 0): 1, ('a', 2, 0): {3}, ('g', 'b', 0): 4, ('a', 1, 0, 0): 2}


In [4]:
a1 = flatten(a, dynamic=False, coerce_keys=True)
a2 = flatten(a, dynamic=False, coerce_keys=False)
print(a1)
print(a2)

{'a': [1, [[2]], [{3}]], 'g|b': [4]}
{('a',): [1, [[2]], [{3}]], ('g', 'b'): [4]}


In [5]:
a1 = flatten(a, dynamic=False, coerce_sequence=False, coerce_keys=True)
a2 = flatten(a, dynamic=False, coerce_sequence=False, coerce_keys=False)
print(a1)
print(a2)

{'a': [1, [[2]], [{3}]], 'g|b': [4]}
{('a',): [1, [[2]], [{3}]], ('g', 'b'): [4]}


In [6]:
a3 = flatten(a, dynamic=True, coerce_sequence="list", coerce_keys=False)

In [7]:
from lionfuncs import as_readable_json

data = {
    "name": "John Doe",
    "age": 30,
    "is_student": False,
    "grades": [{"english": 92}, {"math": 85}],
    "address": {"street": "123 Main St", "city": "Anytown"},
}

schema = extract_json_schema(data)
print(as_readable_json(schema))

{
    "type": "object",
    "properties": {
        "name": {
            "type": "string"
        },
        "age": {
            "type": "integer"
        },
        "is_student": {
            "type": "boolean"
        },
        "address": {
            "street": {
                "type": "string"
            },
            "city": {
                "type": "string"
            }
        },
        "grades": {
            "type": "array",
            "items": {
                "oneOf": [
                    {
                        "english": {
                            "type": "integer"
                        }
                    },
                    {
                        "math": {
                            "type": "integer"
                        }
                    }
                ]
            }
        }
    }
}


In [8]:
data1 = {
    "person": {"name": "Alice", "age": 30, "hobbies": ["reading", "swimming"]},
    "is_employee": True,
}

print(as_readable_json(extract_json_schema(data1)))

{
    "type": "object",
    "properties": {
        "is_employee": {
            "type": "boolean"
        },
        "person": {
            "name": {
                "type": "string"
            },
            "age": {
                "type": "integer"
            },
            "hobbies": {
                "type": "array",
                "items": {
                    "type": "string"
                }
            }
        }
    }
}


In [9]:
data2 = {
    "products": [
        {"id": 1, "name": "Laptop", "price": 999.99},
        {"id": 2, "name": "Mouse", "price": 24.99},
    ],
    "total_products": 2,
}

print(as_readable_json(extract_json_schema(data2)))

{
    "type": "object",
    "properties": {
        "total_products": {
            "type": "integer"
        },
        "products": {
            "type": "array",
            "items": {
                "id": {
                    "type": "integer"
                },
                "name": {
                    "type": "string"
                },
                "price": {
                    "type": "number"
                }
            }
        }
    }
}


In [10]:
import re
from lionfuncs.parsers._extract_json_schema import (
    json_schema_to_cfg,
    print_cfg,
    json_schema_to_regex,
)


# Example schemas
schema1 = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "integer"},
        "is_student": {"type": "boolean"},
    },
}

schema2 = {
    "type": "object",
    "properties": {
        "person": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "address": {
                    "type": "object",
                    "properties": {
                        "street": {"type": "string"},
                        "city": {"type": "string"},
                    },
                },
            },
        }
    },
}

schema3 = {
    "type": "object",
    "properties": {
        "users": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "id": {"type": "integer"},
                    "name": {"type": "string"},
                },
            },
        }
    },
}

# Demonstrate examples
for i, schema in enumerate([schema1, schema2, schema3], 1):
    print(f"\nExample {i}:")
    cfg = json_schema_to_cfg(schema)
    print_cfg(cfg)

# Demonstrate regex
regex_pattern = json_schema_to_regex(schema1)
print("\nRegex pattern for schema1:")
print(regex_pattern)

valid_json = '{"name": "John Doe", "age": 30, "is_student": false}'
invalid_json = '{"name": "John Doe", "age": "thirty", "is_student": "yes"}'

print("\nTesting regex:")
print(f"Valid JSON matches: {bool(re.match(regex_pattern, valid_json))}")
print(f"Invalid JSON matches: {bool(re.match(regex_pattern, invalid_json))}")

# File: lion_core/libs/json_schema_utils.py


Example 1:
S -> { PROPS@0 }
PROPS@0 -> 
PROPS@0 -> name@1
PROPS@0 -> PROPS@0 , age@2
PROPS@0 -> PROPS@0 , is_student@3
name@4 -> "name" : VALUE@5
VALUE@5 -> STRING
age@6 -> "age" : VALUE@7
VALUE@7 -> INTEGER
is_student@8 -> "is_student" : VALUE@9
VALUE@9 -> BOOLEAN

Example 2:
S -> { PROPS@0 }
PROPS@0 -> 
PROPS@0 -> person@1
person@2 -> "person" : VALUE@3
VALUE@3 -> { PROPS@4 }
PROPS@4 -> 
PROPS@4 -> name@5
PROPS@4 -> PROPS@4 , address@6
name@7 -> "name" : VALUE@8
VALUE@8 -> STRING
address@9 -> "address" : VALUE@10
VALUE@10 -> { PROPS@11 }
PROPS@11 -> 
PROPS@11 -> street@12
PROPS@11 -> PROPS@11 , city@13
street@14 -> "street" : VALUE@15
VALUE@15 -> STRING
city@16 -> "city" : VALUE@17
VALUE@17 -> STRING

Example 3:
S -> { PROPS@0 }
PROPS@0 -> 
PROPS@0 -> users@1
users@2 -> "users" : VALUE@3
VALUE@3 -> [ ]
VALUE@3 -> [ ITEMS@4 ]
ITEMS@4 -> VALUE@5
ITEMS@4 -> VALUE@5 , ITEMS@4
VALUE@5 -> { PROPS@6 }
PROPS@6 -> 
PROPS@6 -> id@7
PROPS@6 -> PROPS@6 , name@8
id@9 -> "id" : VALUE@10
VALUE@10 